CINE

In [1]:
class Persona:
    lista_personas = []
    def __init__(self, nombre, contacto):
        self.nombre = nombre
        self.contacto = contacto
    
    def registro(self, per1):
        return self.lista_personas.append(per1)
    
    def act_nombre(self, nombre_nuevo):
        self.nombre = nombre_nuevo
        return self.nombre
    
    def act_contacto(self, contacto_nuevo):
        self.contacto = contacto_nuevo
        return self.contacto

class Empleado(Persona):
    def __init__(self, nombre, contacto, rol):
        super().__init__(nombre, contacto)
        self.rol = rol
    
    def agregar_func(self, hora, sala, pelicula):
        nueva_funcion = Funcion(hora, sala, pelicula)
        return nueva_funcion
    
    def agregar_pelicula(self, titulo, genero, duracion, clasificacion):
        nueva_pelicula = Pelicula(titulo, genero, duracion, clasificacion)
        return nueva_pelicula
    
    def agregar_promocion(self, descuento, condicion):
        nueva_promocion = Promocion(descuento, condicion)
        return nueva_promocion

class Usuario(Persona):
    def __init__(self, nombre, contacto):
        super().__init__(nombre, contacto)
        self.reservas_hechas = []
    
    def hacer_reserva(self, funcion, asientos):
        if funcion.verificar_disponibilidad(asientos):
            nueva_reserva = Reserva(self.nombre, funcion, asientos)
            self.reservas_hechas.append(nueva_reserva)
            funcion.ocupar_asientos(asientos)
            return nueva_reserva
        return None
    
    def cancelar_reserva(self, reserva):
        if reserva in self.reservas_hechas:
            reserva.funcion.liberar_asientos(reserva.asientos)
            self.reservas_hechas.remove(reserva)
            return True
        return False
    
    def acceder_promo(self, promocion):
        if promocion.validar_condicion():
            return promocion.aplicar_promo()
        return None

class Espacio:
    def __init__(self, nombre, capacidad):
        self.nombre = nombre
        self.capacidad = capacidad
    
    def desc(self):
        return f"{self.nombre} tiene capacidad para {self.capacidad} personas"

class Sala(Espacio):
    def __init__(self, nombre, capacidad, tipo):
        super().__init__(nombre, capacidad)
        self.tipo = tipo  # 2D, 3D, IMAX
        self.asientos = [[True for _ in range(10)] for _ in range(self.capacidad // 10)]  # True = disponible
        self.funciones = []
    
    def tipo_sala(self):
        disponibles = sum(row.count(True) for row in self.asientos)
        return f"{self.desc()} es una sala: {self.tipo} y tiene {disponibles} asientos disponibles"
    
    def consultar_disponibilidad(self):
        return sum(row.count(True) for row in self.asientos)

class ZonaComida(Espacio):
    def __init__(self, nombre, capacidad, productos):
        super().__init__(nombre, capacidad)
        self.productos = productos
    
    def menu(self):
        return f"Menu: {self.productos}"
    
    def vender_producto(self, producto, cantidad):
        if producto in self.productos and self.productos[producto]["cantidad"] >= cantidad:
            self.productos[producto]["cantidad"] -= cantidad
            return True
        return False

class Reserva:
    def __init__(self, usuario, funcion, asientos):
        self.usuario = usuario
        self.funcion = funcion
        self.asientos = asientos
    
    def confirmar_reserva(self):
        return f"Usuario: {self.usuario}, Función: {self.funcion.pelicula.titulo}, asientos: {self.asientos}"
    
    def cancelar_reserva(self):
        return f"Reserva cancelada para el usuario {self.usuario} en la función {self.funcion.pelicula.titulo}, asientos: {self.asientos}"

class Funcion:
    def __init__(self, hora, sala, pelicula):
        self.hora = hora
        self.sala = sala
        self.pelicula = pelicula
        self.asientos_ocupados = set()
    
    def verificar_disponibilidad(self, asientos):
        return all(asiento not in self.asientos_ocupados for asiento in asientos)
    
    def ocupar_asientos(self, asientos):
        for asiento in asientos:
            self.asientos_ocupados.add(asiento)
    
    def liberar_asientos(self, asientos):
        for asiento in asientos:
            if asiento in self.asientos_ocupados:
                self.asientos_ocupados.remove(asiento)
    
    def consultar_asientos(self):
        asientos_disponibles = self.sala.capacidad - len(self.asientos_ocupados)
        return f"La sala {self.sala.nombre} tiene {asientos_disponibles} asientos disponibles para la película: {self.pelicula.titulo}"

class Pelicula:
    def __init__(self, titulo, genero, duracion, clasificacion):
        self.titulo = titulo
        self.genero = genero
        self.duracion = duracion
        self.clasificacion = clasificacion
    
    def detalles(self):
        return f"La película {self.titulo} de género {self.genero}, clasificación {self.clasificacion} tiene una duración de {self.duracion} minutos"

class Promocion:
    def __init__(self, descuento, condicion):
        self.descuento = descuento
        self.condicion = condicion
    
    def validar_condicion(self):
        return self.condicion
    
    def aplicar_promo(self):
        if self.validar_condicion():
            return self.descuento
        return 0

##EJEMPLO DE FUNCIONAMIENTO

In [2]:
#datos de ejemplo
def crear_datos_ejemplo():
    # Crear salas
    sala_2d = Sala("Sala 1", 100, "2D")
    sala_3d = Sala("Sala 2", 80, "3D")
    sala_imax = Sala("Sala IMAX", 60, "IMAX")
    
    #Crear películas
    avatar = Pelicula("Avatar", "Ciencia Ficción", 180, "PG-13")
    toy_story = Pelicula("Toy Story 4", "Animación", 120, "ATP")
    avengers = Pelicula("Avengers", "Acción", 150, "PG-13")
    
    #Crear funciones
    funciones = [
        Funcion("15:00", sala_2d, avatar),
        Funcion("18:00", sala_2d, toy_story),
        Funcion("20:00", sala_3d, avatar),
        Funcion("17:00", sala_imax, avengers)
    ]
    
    #Crear promociones
    promo_estudiante = Promocion("50% estudiantes", lambda: True)  
    promo_jubilado = Promocion("30% jubilados", lambda: True)
    
    return sala_2d, sala_3d, sala_imax, funciones, [promo_estudiante, promo_jubilado]

#Crear los datos de ejemplo
sala_2d, sala_3d, sala_imax, funciones, promociones = crear_datos_ejemplo()

#Consultar todas las funciones disponibles
print("=== CARTELERA ===")
for funcion in funciones:
    print(f"Película: {funcion.pelicula.titulo}")
    print(f"Hora: {funcion.hora}")
    print(f"Sala: {funcion.sala.nombre} ({funcion.sala.tipo})")
    print(f"Asientos disponibles: {funcion.consultar_asientos()}")
    print("-" * 30)

#Consultar detalles de una película específica
print("\n=== DETALLES DE PELÍCULA ===")
pelicula = funciones[0].pelicula
print(pelicula.detalles())

#Consultar disponibilidad de una sala específica
print("\n=== DISPONIBILIDAD DE SALA ===")
print(sala_imax.tipo_sala())

#Hacer una reserva y verificar
usuario = Usuario("María", "maria@email.com")
funcion_elegida = funciones[0]
asientos = [(1,1), (1,2)]  # Reservar dos asientos juntos

print("\n=== PROCESO DE RESERVA ===")
print(f"Intentando reservar asientos {asientos} para {funcion_elegida.pelicula.titulo}")
reserva = usuario.hacer_reserva(funcion_elegida, asientos)
if reserva:
    print("Reserva exitosa!")
    print(reserva.confirmar_reserva())
else:
    print("No se pudo realizar la reserva")

#Consultar promociones disponibles
print("\n=== PROMOCIONES DISPONIBLES ===")
for promo in promociones:
    print(f"Descuento: {promo.descuento}")

#Verificar reservas de un usuario
print("\n=== RESERVAS DEL USUARIO ===")
for reserva in usuario.reservas_hechas:
    print(f"Película: {reserva.funcion.pelicula.titulo}")
    print(f"Hora: {reserva.funcion.hora}")
    print(f"Asientos: {reserva.asientos}")

#Consultar asientos disponibles después de la reserva
print("\n=== ASIENTOS DISPONIBLES ACTUALIZADOS ===")
print(funcion_elegida.consultar_asientos())

=== CARTELERA ===
Película: Avatar
Hora: 15:00
Sala: Sala 1 (2D)
Asientos disponibles: La sala Sala 1 tiene 100 asientos disponibles para la película: Avatar
------------------------------
Película: Toy Story 4
Hora: 18:00
Sala: Sala 1 (2D)
Asientos disponibles: La sala Sala 1 tiene 100 asientos disponibles para la película: Toy Story 4
------------------------------
Película: Avatar
Hora: 20:00
Sala: Sala 2 (3D)
Asientos disponibles: La sala Sala 2 tiene 80 asientos disponibles para la película: Avatar
------------------------------
Película: Avengers
Hora: 17:00
Sala: Sala IMAX (IMAX)
Asientos disponibles: La sala Sala IMAX tiene 60 asientos disponibles para la película: Avengers
------------------------------

=== DETALLES DE PELÍCULA ===
La película Avatar de género Ciencia Ficción, clasificación PG-13 tiene una duración de 180 minutos

=== DISPONIBILIDAD DE SALA ===
Sala IMAX tiene capacidad para 60 personas es una sala: IMAX y tiene 60 asientos disponibles

=== PROCESO DE RESERVA